# Sampling Algorithmen
In diesem Jupyter Notebook befindet sich die Implementierung von einem Downsampling und einem Upsampling Algorithmus. Hilfsfunktionen und Metriken befinden sich in dem Notebook metrics.ipynb.

In [1]:
import pymongo
import random
%run metrics.ipynb

### Downsampling
Die Methode downsample(collection, protected_attribute) wendet auf einen Datensatz einen Downsampling Algorithmus an. Der Datensatz ist hierbei in Form einer MongoDB Collection zu übergeben. Folgenden Eingabeparameter werden benötigt:

    collection: pymongo Collection die den Datensatz enthält, auf den Downsampling angewendet wird
    protected_attribute: der Name des geschützten Attributes, nach dem Downsampling angewendet werden soll

Der Algorithmus reduziert die Einträge der bevorzugten Kategorie(n), bis alle Kategorien gleich häufig in dem Datensatz vorhanden sind. Welche Dokumente der MongoDB Collection gelöscht werden, wird hierbei zufällig ausgewählt.

In [2]:
def downsample(collection, protected_attribute):
    toReduce = amountDownsample(analyseFrequency(collection, protected_attribute))
    print('reduce', toReduce)
    row_count = collection.count_documents
    deleteIds = []
    
    for t in toReduce:
        allIds = []
        for obj in raw.find({protected_attribute: t[0]}):
            allIds.append(obj["_id"])
            
        for i in range(0, t[1]):
            rn = random.randint(0, len(allIds)-1)
            deleteIds.append(allIds[rn])
            allIds.remove(allIds[rn])

    query = {"_id": {"$in": deleteIds}}
    collection.delete_many(query)

### Upsampling
Die Methode upsample(collection, protected_attribute) wendet auf einen Datensatz einen Upsampling Algorithmus an. Der Datensatz ist hierbei ebenfalls in Form einer MongoDB Collection zu übergeben. Folgenden Eingabeparameter werden benötigt:

    collection: pymongo Collection die den Datensatz enthält, auf den Upsampling angewendet wird
    protected_attribute: der Name des geschützten Attributes, nach dem Upsampling angewendet werden soll

Der Algorithmus vermehrt die Einträge der bevorzugten Kategorie(n), bis alle Kategorien gleich häufig in dem Datensatz vorhanden sind. Welche Dokumente der MongoDB Collection vervielfacht werden wird hierbei zufällig ausgewählt.

In [3]:
def upsample(collection, protected_attribute):
    toInc = amountUpsample(analyseFrequency(collection, protected_attribute))
    print('Increase:', toInc)
    row_count = collection.count_documents
    
    for t in toInc:
        allIds = []
        for obj in raw.find({protected_attribute: t[0]}):
            allIds.append(obj["_id"])
            
        for i in range(0, t[1]):
            rn = random.randint(0, len(allIds)-1)
            copy = collection.find_one({"_id" : allIds[rn]})
            del copy["_id"]
            collection.insert_one(copy)

## Anwendung der Algorithmen
Als Beispiel wenden die beiden Algorithmen mit dem compas Datensatz getestet. Der Datensatz ist ebenfalls in dem Git Repository vorhanden. Es wird lokal eine MongoDB Instanz auf dem Port 27017 aufgesetzt.  
Das folgende Skript beinhaltet eine setUp() Methode, die eine Datenbank mit einer collection anlegt und anschließend den compas Datensatz in diese Collection lädt. Falls bereits eine collection angelegt wurde wird diese gelöscht.

In [4]:
import csv
import codecs

client = pymongo.MongoClient('localhost', 27017)
db = client.compas
compas = db.compas

def load_csv(path):
    with codecs.open(path,"r", "utf-8") as file:
        return [row for row in csv.DictReader(file)]

def load_and_insert_csv(path, collection):
    with codecs.open(path,"r", "utf-8") as file:
        data = [row for row in csv.DictReader(file)]
        collection.insert_many(data)
        print("CSV successfully loaded into collection!")

def setUp():
    print("Resetting MongoDB Collection...")
    db.drop_collection("compas")
    load_and_insert_csv("Datensatz/compas-scores-raw.csv", compas)

setUp()

Resetting MongoDB Collection...
CSV successfully loaded into collection!


### Anwendungsbeispiel von Downsampling
Zuerst werden zwei Metriken ausgeführt, die zeigen, wie stark die Verteilung der Kategorien eines Attributs in dem Datensatz vertreten sind. Die erste Metrik zeigt den prozentualen Anteil und die zweite Metrik die genaue Anzahl der Kategorie im Datensatz. Als Beispiel werden die Metriken einmal mit dem Geschlecht und einmal mit der Ethnischen Gruppe ausgeführt.  
Die Anwendung der Metriken zeigt, dass Männer öfter in dem Datensatz vertreten sind als Frauen.

In [5]:
print("Prozentanteil der Einträge nach Geschlecht\n", analysePercentage(compas, 'Sex_Code_Text'))
print("\n\nAnzahl der Einträge nach Geschlecht\n", analyseFrequency(compas, 'Sex_Code_Text'))

print("\n\nProzentanteil der Einträge nach Ethnischer Gruppe\n", analysePercentage(compas, 'Ethnic_Code_Text'))
print("\n\nAnzahl der Einträge nach Ethnischer Gruppe\n", analyseFrequency(compas, 'Ethnic_Code_Text'))

Prozentanteil der Einträge nach Geschlecht
 [('Female', 0.21907203786795523), ('Male', 0.7809279621320447)]


Anzahl der Einträge nach Geschlecht
 [('Female', 13329), ('Male', 47514)]


Prozentanteil der Einträge nach Ethnischer Gruppe
 [('African-Am', 0.0008382229673093043), ('African-American', 0.44406094374044675), ('Arabic', 0.0012326808342783887), ('Asian', 0.005325181204082639), ('Caucasian', 0.3580198215078152), ('Hispanic', 0.14368127804348899), ('Native American', 0.0035994280360928947), ('Oriental', 0.0006409940338247621), ('Other', 0.04260144963266111)]


Anzahl der Einträge nach Ethnischer Gruppe
 [('African-Am', 51), ('African-American', 27018), ('Arabic', 75), ('Asian', 324), ('Caucasian', 21783), ('Hispanic', 8742), ('Native American', 219), ('Oriental', 39), ('Other', 2592)]


Jetzt wird der Downsampling Algorithmus auf dem Datensatz angewendet. Als Beispiel nehmen wir hierbei das Attribut des Geschlechts als Downsampling Attribut. Der Algorithmus zeigt an, welche Kategorie um wie viele Datensätze er reduziert.

In [6]:
downsample(compas, "Sex_Code_Text")

reduce [('Male', 34185)]


Nachdem der Algorithmus angewendet wurde, kann man den Metriken entnehmen, dass beide Geschlechter nun gleich häufig im Datensatz vertreten sind.

In [7]:
print("Prozentanteil der Einträge nach Geschlecht\n", analysePercentage(compas, 'Sex_Code_Text'))
print("\n\nAnzahl der Einträge nach Geschlecht\n", analyseFrequency(compas, 'Sex_Code_Text'))

Prozentanteil der Einträge nach Geschlecht
 [('Female', 0.5), ('Male', 0.5)]


Anzahl der Einträge nach Geschlecht
 [('Female', 13329), ('Male', 13329)]


### Anwendungsbeispiel von Upsampling
Bevor Upsampling auf dem Datensatz angewendet wird, wird zuerst die Collection mit der setUp() Methode zurückgesetzt. Anschließend wird der Upsampling Algorithmus mit dem geschützten Attribut Geschlecht ausgeführt.

In [8]:
setUp()
upsample(compas, "Sex_Code_Text")

Resetting MongoDB Collection...
CSV successfully loaded into collection!
Increase: [('Female', 34185)]


Nach Anwendung des Algorithmus deuten die Metriken erneut auf eine gleiche Verteilung des Geschlechts im Datensatz hin. Der Unterschied zum Downsampling ist bei der Anzahl der Einträge zu erkennen.

In [9]:
print("Prozentanteil der Einträge nach Geschlecht\n", analysePercentage(compas, 'Sex_Code_Text'))
print("\n\nAnzahl der Einträge nach Geschlecht\n", analyseFrequency(compas, 'Sex_Code_Text'))

Prozentanteil der Einträge nach Geschlecht
 [('Female', 0.5), ('Male', 0.5)]


Anzahl der Einträge nach Geschlecht
 [('Female', 47514), ('Male', 47514)]
